In [ ]:
# Restart Kernel
!pip install -qq -U bitsandbytes

In [ ]:
import pandas as pd
import numpy as np

import shutil
from pathlib import Path
import os
import random
import re
import json

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2
from PIL import Image
from tqdm import tqdm
from datasets import load_dataset

import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import nn, optim

from peft import PeftModel
from transformers import (AutoModelForCausalLM, Blip2Processor, Blip2Model, BlipImageProcessor, AutoTokenizer, BitsAndBytesConfig,
    Trainer, TrainingArguments, default_data_collator, TrainerCallback)

from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

from datasets import load_dataset



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

save_path = '/kaggle/working/'

In [ ]:
class BLIP2ForPhi(nn.Module):
    def __init__(self, vision_model, q_former, phi_model, query_tokens):
        super().__init__()
        self.vision_model = vision_model
        self.q_former = q_former
        self.projection = nn.Linear(q_former.config.hidden_size, phi_model.config.hidden_size)
        self.phi_model = phi_model
        self.query_tokens = query_tokens
        

        print("Freezing vision_model and phi_model parameters...")
        for param in self.vision_model.parameters():
            param.requires_grad = False
        for param in self.phi_model.parameters():
            param.requires_grad = False
        
        print("Training q_former and projection layer...")
        for param in self.q_former.parameters():
            param.requires_grad = True
        for param in self.projection.parameters():
            param.requires_grad = True

    
    def forward(self, pixel_values, input_ids, attention_mask, labels=None):
        image_embeds = self.vision_model(pixel_values).last_hidden_state

        batch_size = image_embeds.shape[0]
        qformer_query_embeds = self.query_tokens.expand(batch_size, -1, -1)

        
        image_attention_mask = torch.ones(image_embeds.size()[:-1], dtype=torch.long, device=image_embeds.device)
        query_outputs = self.q_former(
            query_embeds=qformer_query_embeds,
            encoder_hidden_states=image_embeds,
            encoder_attention_mask=image_attention_mask
        )[0]

        projected_query = self.projection(query_outputs)

        text_embeds = self.phi_model.get_input_embeddings()(input_ids)
        inputs_embeds = torch.cat([projected_query, text_embeds], dim=1)
        
        # 입력 ID의 attention_mask와 쿼리의 attention_mask를 결합
        query_attention_mask = torch.ones(projected_query.size()[:-1], dtype=torch.long, device=projected_query.device)
        combined_attention_mask = torch.cat([query_attention_mask, attention_mask], dim=1)

        
        outputs = self.phi_model(
            inputs_embeds=inputs_embeds,
            attention_mask=combined_attention_mask,
            labels=labels, 
        )
        return outputs

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)


image_processor = BlipImageProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
if phi_tokenizer.pad_token is None:
    phi_tokenizer.pad_token = phi_tokenizer.eos_token

blip2_model = Blip2Model.from_pretrained("Salesforce/blip2-opt-2.7b")

vision_model = blip2_model.vision_model
q_former = blip2_model.qformer
query_tokens = blip2_model.query_tokens

phi_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-1_5",
    quantization_config=quantization_config,
    trust_remote_code=True,
)

In [ ]:
model = BLIP2ForPhi(vision_model, q_former, phi_model, query_tokens)

In [ ]:
class Stage1Dataset(Dataset):
    def __init__(self, hf_dataset, image_processor, tokenizer, num_query_tokens=32, max_length=128):
        self.dataset = hf_dataset
        self.image_processor = image_processor
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.num_query_tokens = num_query_tokens
        

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item['image'].convert("RGB")
        captions = item['answer']
        caption = captions[torch.randint(0, len(captions), (1,)).item()].replace('\n', ' ').strip()

        
        pixel_values = self.image_processor(image, return_tensors="pt").pixel_values

        inputs = self.tokenizer(
            caption,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        
        text_labels = inputs.input_ids.clone()
        text_labels[text_labels == self.tokenizer.pad_token_id] = -100
        
        query_labels = torch.full((1, self.num_query_tokens), -100)

        combined_labels = torch.cat([query_labels, text_labels], dim=1)

        return {
            "pixel_values": pixel_values.squeeze(),
            "input_ids": inputs.input_ids.squeeze(),   
            "attention_mask": inputs.attention_mask.squeeze(),
            "labels": combined_labels.squeeze()
        }

raw_dataset = load_dataset("lmms-lab/COCO-Caption")['val']

split_dataset = raw_dataset.train_test_split(test_size=0.2)
train_raw_dataset = split_dataset['train']
eval_raw_dataset = split_dataset['test']

train_dataset = Stage1Dataset(train_raw_dataset, image_processor, phi_tokenizer)
valid_dataset = Stage1Dataset(eval_raw_dataset, image_processor, phi_tokenizer)
train_debug = Subset(train_dataset, indices=range(50))
valid_debug = Subset(valid_dataset, indices=range(50))


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import get_cosine_schedule_with_warmup
import os
from pathlib import Path
from collections import deque

class CustomTrainer:
    
    def __init__(self, model: nn.Module, optimizer, tokenizer, train_dataset, val_dataset=None, batch_size=8, save_dir="./checkpoints"):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = model.to(self.device)
        self.optimizer = optimizer
        self.tokenizer = tokenizer
        
        self.train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
        self.val_dataloader = DataLoader(val_dataset, batch_size=batch_size, num_workers=4, pin_memory=True) if val_dataset else None
        
        self.scaler = torch.cuda.amp.GradScaler() # 혼합 정밀도 학습용 스케일러
        self.scheduler = None # train 메서드 내에서 설정

        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(parents=True, exist_ok=True)
        print(f"Using device: {self.device}")

    def _forward_step(self, batch: dict, return_preds: bool = False):
       
        inputs = {k: v.to(self.device) for k, v in batch.items()}
        
        with torch.autocast(device_type='cuda', dtype=torch.float16):
            outputs = self.model(**inputs)
            loss = outputs.loss

        if return_preds:
            pred_ids = torch.argmax(outputs.logits, dim=-1)
            decoded_preds = self.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
            
            labels = inputs['labels'].clone()
            labels[labels == -100] = self.tokenizer.pad_token_id
            decoded_labels = self.tokenizer.batch_decode(labels, skip_special_tokens=True)
            return loss, decoded_preds, decoded_labels

        return loss

    def train(self, num_epochs: int, resume_from_checkpoint: str = None):
      
        total_steps = len(self.train_dataloader) * num_epochs
        warmup_steps = int(0.1 * total_steps)
        
        self.scheduler = get_cosine_schedule_with_warmup(
            optimizer=self.optimizer,
            num_warmup_steps=warmup_steps,
            num_training_steps=total_steps,
        )

        if resume_from_checkpoint:
            start_epoch = self.load_checkpoint(resume_from_checkpoint)

        for epoch in range(start_epoch, num_epochs):
            self.model.train()
            epoch_loss = 0

            progress_bar = tqdm(self.train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} - Training")
            for batch in progress_bar:
                self.optimizer.zero_grad()
                
                loss = self._forward_step(batch)
                
                self.scaler.scale(loss).backward()
                self.scaler.step(self.optimizer)
                self.scaler.update()
                self.scheduler.step()
                
                epoch_loss += loss.item()
                progress_bar.set_postfix(loss=loss.item(), lr=self.scheduler.get_last_lr()[0])
            
            avg_train_loss = epoch_loss / len(self.train_dataloader)
            print(f"Epoch {epoch+1} | Average Train Loss: {avg_train_loss:.4f}")

            if self.val_dataloader:
                avg_val_loss = self.evaluate(epoch)
            if epoch == num_epochs - 1:
                self.save_checkpoint(epoch, avg_train_loss, avg_val_loss)

    def evaluate(self, epoch: int):
        
        self.model.eval()
        total_loss = 0
        
        last_n_samples = 5
        last_preds = deque(maxlen=last_n_samples)
        last_labels = deque(maxlen=last_n_samples)

        with torch.no_grad():
            progress_bar = tqdm(self.val_dataloader, desc=f"Epoch {epoch+1} - Evaluating")
            for batch in progress_bar:
                loss, decoded_preds, decoded_labels = self._forward_step(batch, return_preds=True)
                total_loss += loss.item()
                
                last_preds.extend(decoded_preds)
                last_labels.extend(decoded_labels)

        avg_val_loss = total_loss / len(self.val_dataloader)
        print(f"\n--- Validation Results for Epoch {epoch+1} ---")
        print(f"Average Validation Loss: {avg_val_loss:.4f}")
        
        print("\n--- Last 5 Sample Predictions ---")
        for pred, label in zip(last_preds, last_labels):
            print(f"🔵 Pred:  {pred.strip()}")
            print(f"🟢 Label: {label.strip()}")
        print("---------------------------------------\n")

        return avg_val_loss

    def save_checkpoint(self, epoch: int, train_loss: float, val_loss: float):
        """모델의 체크포인트를 저장합니다."""
        save_path = self.save_dir / f"epoch_{epoch+1}"
        save_path.mkdir(parents=True, exist_ok=True)
        
        trainable_state_dict = {k: v for k, v in self.model.state_dict().items() if v.requires_grad}
        
        checkpoint = {
            "epoch": epoch + 1,
            "model_state_dict": trainable_state_dict,
            "optimizer_state_dict": self.optimizer.state_dict(),
            "scaler_state_dict": self.scaler.state_dict(),
            "scheduler_state_dict": self.scheduler.state_dict(),
            "train_loss": train_loss,
            "val_loss": val_loss,
        }
        torch.save(checkpoint, save_path / "checkpoint.pt")
        print(f"✅ Checkpoint saved to {save_path}")

    def load_checkpoint(self, checkpoint_path: str):
       
        checkpoint = torch.load(checkpoint_path, map_location=self.device)
        
        
        self.model.load_state_dict(checkpoint['model_state_dict'], strict=False)
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scaler.load_state_dict(checkpoint['scaler_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        
        start_epoch = checkpoint['epoch']
        print(f"✅ Checkpoint loaded. Resuming from epoch {start_epoch}")
        return start_epoch

In [ ]:
trainable_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.AdamW(trainable_params, lr=1e-4, weight_decay=0.01)

trainer = CustomTrainer(
    model=model,
    optimizer=optimizer,
    tokenizer=phi_tokenizer,
    train_dataset=train_dataset,
    val_dataset=valid_dataset,
    batch_size=4
)

checkpoint_path = "/kaggle/input/blip2_epoch11/pytorch/default/1/epoch_11/checkpoint.pt"

trainer.train(num_epochs=13, resume_from_checkpoint=checkpoint_path) 

In [ ]:
shutil.make_archive("/kaggle/working/checkpoints", 'zip', "/kaggle/working/checkpoints")